In [1]:
!pip install opendatasets
!pip install pandas

In [2]:
import opendatasets as od
import pandas

od.download(
    "https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: omidmohebi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset


100%|██████████| 195M/195M [00:02<00:00, 91.3MB/s]


In [3]:
import pandas as pd

data = pd.read_csv('movielens-20m-dataset/rating.csv')
data['userId'] -= 1
data = data.drop(columns = ['timestamp'])
data.head()

,userId,movieId,rating
0,0,2,3.5
1,0,29,3.5
2,0,32,3.5
3,0,47,3.5
4,0,50,3.5


In [4]:
import numpy as np

total_movie = set(data['movieId'])
movie2idx = {}
counter = 0
for movieId in total_movie:
  movie2idx[movieId] = counter
  counter += 1
data.head()
raw_data = data.to_numpy()

for row in range(len(raw_data)):
  raw_data[row, 1] = movie2idx[raw_data[row, 1]]

In [5]:
stars = set(data['rating'])
star2idx = {}
counter = 0
for star in stars:
  star2idx[star] = counter
  counter += 1
for row in range(len(stars)):
  raw_data[row, 2] = star2idx[raw_data[row, 2]]
raw_data = raw_data.astype(int)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, _, _ = train_test_split(raw_data, raw_data, test_size=0.2, random_state=42)

In [7]:
from scipy.sparse import lil_matrix

sparse_matrix_train = lil_matrix((max(raw_data[:,0]) + 1, max(raw_data[:,1]) + 1))
sparse_matrix_test = lil_matrix((max(raw_data[:,0]) + 1, max(raw_data[:,1]) + 1))

for row in X_train:
  sparse_matrix_train[row[0], row[1]] = row[2]
for row in X_test:
  sparse_matrix_test[row[0], row[1]] = row[2]

In [ ]:
def one_hot_encode(X, K):
    N, D = X.shape
    Y = np.zeros((N, D, K))
    for n, d in zip(*X.nonzero()):
        Y[n, d, int(X[n, d])] = 1
    return Y

In [ ]:
def one_hot_mask_encode(X, K):
    N, D = X.shape
    Y = np.zeros((N, D, K))
    for n, d in zip(*X.nonzero()):
        Y[n, d, :] = 1
    return Y

In [10]:
def prob2rate(prob):
  return prob.dot((np.arange(10) + 1))

In [11]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

def vis_dot(V, W):
  return tf.tensordot(V, W, axes = [[1, 2], [0, 1]])

def hid_dot(H, W):
  return tf.tensordot(H, W, axes = [[1], [2]])

In [ ]:
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

class Restricted_Boltzmann_Machine(object):
  def __init__(self, D, M, K):
    self.D = D
    self.M = M
    self.K = K
    self.build(D, M, K)

  def build(self, D, M, K):
    self.W = tf.Variable(tf.random.normal(shape = (D, K, M)) * np.sqrt(2.0 / M))
    self.C = tf.Variable(np.zeros(M).astype(np.float32))
    self.B = tf.Variable(np.zeros((D, K)).astype(np.float32))

    self.Xin = tf.compat.v1.placeholder(tf.float32, shape = (None, D, K))
    self.mask = tf.compat.v1.placeholder(tf.float32, shape = (None, D, K))

    V = self.Xin
    ph_gy = tf.nn.sigmoid(vis_dot(V, self.W) + self.C)
    self.ph_gy = ph_gy

    R = tf.random.uniform(shape = tf.shape(ph_gy))
    H = tf.cast(R < ph_gy, tf.float32)
    logits = hid_dot(H, self.W) + self.B
    cat_distribution = tf.compat.v1.distributions.Categorical(logits)
    sample = cat_distribution.sample()
    sample = tf.one_hot(sample, K)
    sample = sample * self.mask

    objective = tf.reduce_mean(self.free_energy(self.Xin)) - tf.reduce_mean(self.free_energy(sample))
    self.optimazer = tf.compat.v1.train.AdamOptimizer(1e-2).minimize(objective)

    logits = self.forward_logits(self.Xin)
    self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(tf.stop_gradient(self.Xin), logits))

    self.visible_output = self.forward_output(self.Xin)

    initop = tf.compat.v1.global_variables_initializer()
    self.session = tf.compat.v1.Session()
    self.session.run(initop)

  def fit(self, X_train, mask_train, X_test, mask_test, epochs = 10, batch_size = 256, show_figure = True):
    N, D = X_train.shape
    batches = N // batch_size
    train_cost = []
    test_cost = []

    for i in range(epochs):
      print('epochs: ', i)
      X_train, mask_train, X_test, mask_test = shuffle(X_train, mask_train, X_test, mask_test)

      for j in range(batches):
        x = X_train[j * batch_size:(j * batch_size + batch_size)].toarray()
        m = mask_train[j * batch_size:(j * batch_size + batch_size)].toarray()

        b_one_hot = one_hot_encode(x, self.K)
        m_one_hot = one_hot_mask_encode(m, self.K)

        _, c = self.session.run((self.optimazer, self.cost), {self.Xin: b_one_hot, self.mask: m_one_hot})

        if j % 100 == 0:
          print(c)

      train_sum_sqr_err = 0
      test_sum_sqr_err = 0
      train_n = 0
      test_n = 0

      for j in range(batches):
        x = X_train[j * batch_size:(j * batch_size + batch_size)].toarray()
        m = mask_train[j * batch_size:(j * batch_size + batch_size)].toarray()

        xoh = one_hot_encode(x, self.K)
        probs = self.get_visible(xoh)
        xhat = prob2rate(probs)
        train_sum_sqr_err += (m * (xhat - x) * (xhat - x)).sum()
        train_n += m.sum()

        xt = X_test[j * batch_size:(j * batch_size + batch_size)].toarray()
        mt = mask_test[j * batch_size:(j * batch_size + batch_size)].toarray()

        test_sum_sqr_err = (mt * (xhat - xt) * (xhat - xt)).sum()
        test_n += mt.sum()
      c = train_sum_sqr_err / train_n
      ct = test_sum_sqr_err / test_n

      print('train mse: ', c)
      print('test mse: ', ct)

      train_cost.append(c)
      test_cost.append(ct)

    if show_figure:
      plt.plot(train_cost, 'train mse')
      plt.plot(test_cost, 'test mse')
      plt.legend()
      plt.show()

  def free_energy(self, V):
    f_term = -tf.reduce_sum(vis_dot(V, self.B))
    s_term = -tf.reduce_sum(tf.nn.softplus(vis_dot(V, self.W) + self.C), 1)
    return f_term + s_term

  def forward_hidden(self, X):
    return tf.nn.sigmoid(vis_dot(X, self.W) + self.C)

  def forward_logits(self, X):
    z = self.forward_hidden(X)
    return hid_dot(z, self.W) + self.B

  def forward_output(self, X):
    return tf.nn.softmax(self.forward_logits(X))

  def get_visible(self, X):
    return self.session.run(self.visible_output, {self.Xin: X})


In [ ]:
mask_train = (sparse_matrix_train > 0) * 1.0
mask_test = (sparse_matrix_test > 0) * 1.0

N, M = sparse_matrix_train.shape
rbm = Restricted_Boltzmann_Machine(M, 50, 10)
rbm.fit(sparse_matrix_train, mask_train, sparse_matrix_test, mask_test)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use `tf.random.categorical` instead.


epochs:  0
0.008471146
0.004740864
0.0057666074
0.006643155
0.005626987
0.004948587
train mse:  1.7225339892914977
test mse:  0.002863464319621007
epochs:  1
0.005052323
0.004800773
0.0055335783
0.0044070603
0.005497138
0.0046447925
train mse:  1.6586920330667398
test mse:  0.0027551624562209965
epochs:  2
0.005049088
0.0051679113
0.0044759214
0.004829394
0.0060880245
0.004263644
train mse:  1.6449854422722727
test mse:  0.0033929115813869956
epochs:  3
0.0039483504
0.0053124316
0.004440118
0.0044928584
0.0047837896
0.0045728893
train mse:  1.6178950599819866
test mse:  0.0030436126630754338
epochs:  4
0.0042410544
0.0046846005
0.0038840922
0.0048330976
0.0041909153


In [ ]:
mask_train = (sparse_matrix_train > 0) * 1.0
mask_test = (sparse_matrix_test > 0) * 1.0

N, M = sparse_matrix_train.shape
rbm = Restricted_Boltzmann_Machine(M, 50, 10)
rbm.fit(sparse_matrix_train, mask_train, sparse_matrix_test, mask_test)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use `tf.random.categorical` instead.


epochs:  0
0.012882293
0.0059807673
0.00517317
0.0054936763
0.0057072043
0.004644237
train mse:  1.7317691608777692
test mse:  0.003404992487442568
epochs:  1
0.0051958133
0.0058200327
0.005759504
0.0041920743
0.004540045
0.0047820755
train mse:  1.672546363751807
test mse:  0.003533659753156751
epochs:  2
0.004530783
0.004725766
0.004907881
0.0045330296
0.0044788853
0.0039277133
train mse:  1.6454116906916945
test mse:  0.003198443304884623
epochs:  3
0.004274084
0.004840111
0.005194969
0.0052414844
0.0058428254
0.0044135507
train mse:  1.6196608007089581
test mse:  0.002957293645996951
epochs:  4
0.005595419
0.0041198195
0.004340069
0.0040098405
0.0048649088
0.0045699324
train mse:  1.60947688895605
test mse:  0.0030081611388833042
epochs:  5
0.0042826603
0.0040569813
0.0044071553
0.004708439
0.004254772
0.0040293303
train mse:  1.6029149439910635
test mse:  0.003587898972151615
epochs:  6
0.004930839
0.004594511
